# FACT-UVA: Man is to Programmer as Woman is to Homemaker?

## Links

Debiaswe: https://github.com/tolga-b/debiaswe
Lipstick: https://github.com/gonenhila/gender_bias_lipstick

### How to get the GoogleNews word2vec embeddings:
Download it directly from the official [website](https://code.google.com/archive/p/word2vec/) or clone [this github repo](https://github.com/mmihaltz/word2vec-GoogleNews-vectors). Place the downloaded **.bin** file in the embeddings folder.

### How to get the Glove embeddings:
Go to the official [website](https://nlp.stanford.edu/projects/glove/). Download **glove.840B.300d.zip**. Place the downloaded **.txt** file in the embeddings folder.

## Debiasing Word Embeddings

In [5]:
# Debias word2vec embeddings
!python3 code/main.py --debias_o_em=embeddings/debiased_word2vec.bin --i_em=embeddings/GoogleNews-vectors-negative300.bin --bias_o_em=embeddings/bias_word2vec.bin --def_fn=data/definitional_pairs.json --g_words_fn=data/gender_specific_full.json --eq_fn=data/equalize_pairs.json

Namespace(bias_o_em='embeddings/bias_word2vec.bin', debias_o_em='embeddings/debiased_word2vec.bin', def_fn='data/definitional_pairs.json', em_limit=50000, eq_fn='data/equalize_pairs.json', g_words_fn='data/gender_specific_full.json', i_em='embeddings/GoogleNews-vectors-negative300.bin', o_ext='bin')
*** Reading data from embeddings/GoogleNews-vectors-negative300.bin
/home/yovchev/miniconda3/envs/test/lib/python3.5/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
Number of words:  26391
Saving biased vectors to file...
Debiasing...
Saving to file...


Done!



In [ ]:
#